In [47]:
import warnings
import os
from IPython.display import Audio, display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import FactorAnalysis
from sklearn import preprocessing
from scipy.stats import norm
warnings.simplefilter('ignore')
%matplotlib inline

In [4]:
train_fl = pd.read_csv('./train_files.csv', header=None)
test_fl = pd.read_csv('./test_files.csv', header=None)
valid_fl = pd.read_csv('./val_files.csv', header=None)

In [5]:
data = pd.read_csv('./data/list_mfccs.csv')

In [8]:
cols = ['mfcc_%s'%i for i in range(1, 40)]
scaler = preprocessing.StandardScaler()
scaled_df = scaler.fit_transform(data[cols])


In [9]:
data_scaled = data.copy()
data_scaled[cols] = pd.DataFrame(scaled_df)

In [10]:
train = data_scaled[data.filename.map(lambda x: x in set(train_fl[1]))]
test = data_scaled[data.filename.map(lambda x: x in set(test_fl[1]))]
val = data_scaled[data.filename.map(lambda x: x in set(valid_fl[1]))]

In [48]:
train['idx'] = list(train.index)
test['idx'] = list(test.index)
val['idx'] = list(val.index)

In [22]:
train.head()

,filename,class,lbl,mfcc_1,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,...,mfcc_31,mfcc_32,mfcc_33,mfcc_34,mfcc_35,mfcc_36,mfcc_37,mfcc_38,mfcc_39,idx
0,blues/blues.00000.au,blues,1,-0.724054,-0.500629,-0.165197,-0.224717,0.344881,0.522743,-1.350355,...,0.028203,-0.075622,-0.863715,-1.345201,-0.072421,0.18751,0.406762,0.196529,-0.217599,0
1,blues/blues.00000.au,blues,1,-0.525030,-0.067544,-0.145980,0.647703,0.637306,0.695705,-0.702556,...,0.028203,-0.075622,-0.863715,-1.345201,-0.072421,0.18751,0.268448,0.196529,-0.217599,1
2,blues/blues.00000.au,blues,1,-0.590512,-0.206208,-0.388126,0.741486,0.722044,0.497295,-0.975393,...,0.028203,-0.075622,-0.863715,-1.345201,-0.072421,0.18751,0.325000,0.196529,-0.217599,2
3,blues/blues.00000.au,blues,1,-0.498473,-0.023238,-0.278193,0.395156,-0.000983,0.510387,-1.066385,...,0.028203,-0.075622,-0.863715,-1.345201,-0.072421,0.18751,0.239851,0.196529,-0.217599,3
4,blues/blues.00000.au,blues,1,-0.456160,-0.403885,-0.030121,1.040700,0.450166,1.400972,0.105636,...,0.028203,-0.075622,-0.863715,-1.345201,-0.072421,0.18751,0.191897,0.196529,-0.217599,4


In [29]:
classes = set( ['classical', 'jazz', 'metal'])

In [34]:
tmp_data = data[data['class'].map(lambda x: x in classes)]

In [111]:
tmp_train = train[train['class'] is in classes]
tmp_test = test[test['class']is in classes]
tmp_val = val[val['class']is in classes]

In [114]:
gmm = GaussianMixture(n_components=200, random_state=45, verbose=1)

In [115]:
%%time
gmm.fit(tmp_train[cols].as_matrix())

Initialization 0
  Iteration 0
  Iteration 10
  Iteration 20
  Iteration 30
  Iteration 40
  Iteration 50
  Iteration 60
  Iteration 70
Initialization converged: True
Wall time: 3h 38min 44s


GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
        means_init=None, n_components=200, n_init=1, precisions_init=None,
        random_state=45, reg_covar=1e-06, tol=0.001, verbose=1,
        verbose_interval=10, warm_start=False, weights_init=None)

In [116]:
pred_train = gmm.predict_proba(tmp_train[cols].as_matrix())

In [117]:
pred_test = gmm.predict_proba(tmp_test[cols].as_matrix())
pred_val = gmm.predict_proba(tmp_val[cols].as_matrix())

In [228]:
mn = gmm.means_.reshape(1, -1)
eps = 1e-30

In [150]:
#... цикл по трекам для train...
M = []
for i in range(tmp_train.shape[0] // 2000):
    eps
    tmp = (pred_train[i*2000:(i+1)*2000, :, None]
           * tmp_train[cols].as_matrix()[i*2000:(i+1)*2000, None, :]
          ).sum(axis=0) + eps
    tmp = tmp / (pred_train[i*2000:(i+1)*2000].sum(axis=0)[:, None] + eps*200)
    M.append(tmp.reshape(1, -1))

In [151]:
#... цикл по трекам...
M_test = []
for i in range(tmp_test.shape[0] // 2000):

    tmp = (pred_test[i*2000:(i+1)*2000, :, None]
           * tmp_test[cols].as_matrix()[i*2000:(i+1)*2000, None, :]
          ).sum(axis=0) + eps
    s = pred_test[i*2000:(i+1)*2000].sum(axis=0)[:, None] + eps * 200
    tmp = tmp / s
    
        
    M_test.append(tmp.reshape(1, -1))

In [152]:
#... цикл по трекам для валидации...
M_val = []
for i in range(tmp_val.shape[0] // 2000):
    tmp = (pred_val[i*2000:(i+1)*2000, :, None]
           * tmp_val[cols].as_matrix()[i*2000:(i+1)*2000, None, :]
          ).sum(axis=0) + eps
    tmp = tmp / (pred_val[i*2000:(i+1)*2000].sum(axis=0)[:, None] + eps*200)
    M_val.append(tmp.reshape(1, -1))

In [153]:
M = np.vstack(M)
M_test = np.vstack(M_test)
M_val = np.vstack(M_val)

In [197]:
labels = [tmp_train['lbl'].iloc[i*2000] for i in range(tmp_train.shape[0]// 2000)]
labels_test = [tmp_test['lbl'].iloc[i*2000] for i in range(tmp_test.shape[0]// 2000)]
labels_val = [tmp_val['lbl'].iloc[i*2000] for i in range(tmp_val.shape[0]// 2000)]

### Эксперименты

In [187]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn.linear_model import SGDClassifier

### №1

In [154]:
fa = FactorAnalysis(n_components=400)

In [179]:
%%%time
fa.fit(M)

Wall time: 5.96 s


FactorAnalysis(copy=True, iterated_power=3, max_iter=1000, n_components=400,
        noise_variance_init=None, random_state=0, svd_method='randomized',
        tol=0.01)

In [182]:
t = fa.get_covariance()[:400]

In [183]:
w = np.linalg.inv(t.dot(t.T)).dot(t.dot((M - mn).T))

In [184]:
w_test = np.linalg.inv(t.dot(t.T)).dot(t.dot((M_test - mn).T))
w_val = np.linalg.inv(t.dot(t.T)).dot(t.dot((M_val - mn).T))

In [189]:
sgd_clf = SGDClassifier(max_iter=100, loss='log', penalty='elasticnet')

In [201]:
%time
model = OneVsOneClassifier(sgd_clf).fit(w.T, labels)

Wall time: 0 ns


In [204]:
model.predict(w_test.T)

array([2, 6, 2, 2, 6, 2, 2, 2, 2, 2, 6, 6, 6, 6, 6, 2, 6, 6, 2, 6, 6, 7,
       7, 7, 7, 7, 7, 7, 7, 7], dtype=int64)

In [208]:
model.score(w_test.T, labels_test)

0.8333333333333334

### №2

In [222]:
fa = FactorAnalysis(n_components=800)

In [223]:
%%%time
fa.fit(M)

Wall time: 8.32 s


FactorAnalysis(copy=True, iterated_power=3, max_iter=1000, n_components=800,
        noise_variance_init=None, random_state=0, svd_method='randomized',
        tol=0.01)

In [224]:
t = fa.get_covariance()[:800]

In [229]:
w2 = np.linalg.inv(t.dot(t.T)).dot(t.dot((M - mn).T))

In [230]:
w_test2 = np.linalg.inv(t.dot(t.T)).dot(t.dot((M_test - mn).T))
w_val2 = np.linalg.inv(t.dot(t.T)).dot(t.dot((M_val - mn).T))

In [231]:
sgd_clf = SGDClassifier(max_iter=100, loss='log', penalty='elasticnet')

In [232]:
%time
model = OneVsOneClassifier(sgd_clf).fit(w2.T, labels)

Wall time: 0 ns


In [233]:
model.predict(w_test2.T)

array([2, 2, 2, 2, 2, 6, 2, 2, 2, 2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7], dtype=int64)

In [235]:
model.score(w_test2.T, labels_test)

0.9666666666666667

### №3

In [243]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

lda = LinearDiscriminantAnalysis(n_components=200)
X_lda = lda.fit(w2.T, labels).transform(w2.T)

print(lda.explained_variance_ratio_)

'''for i in range(tmp_test.shape[0] // 2000):
    X_t.append(get_w(w, i))
    y_t.append(tmp_test['lbl'][i * 2000])
'''
x_lda_test = lda.transform(w_test2.T)

[0.79816789 0.20183211]


In [244]:
sgd_clf2 = SGDClassifier(max_iter=100, loss='log', penalty='elasticnet')

In [245]:
%time
model2 = OneVsOneClassifier(sgd_clf2).fit(X_lda, labels)

Wall time: 0 ns


In [248]:
model2.predict(x_lda_test)

array([6, 2, 6, 2, 7, 2, 6, 2, 2, 2, 6, 2, 6, 6, 6, 6, 6, 2, 6, 6, 2, 6,
       7, 7, 7, 7, 2, 7, 7, 2], dtype=int64)

In [249]:
model2.score(x_lda_test, labels_test)

0.6666666666666666